### [함수] 1개 행정동 내 여러 관측소 매핑 함수

In [0]:
def multi_st(num):
    df_return = tm_all
    for i in n_hdong.loc[no_dong[num],:].values:  # 여기서 no_dong[_] 안의 숫자만 바꿔가면서 돌려주면 됨
        filepath = 'C:\Python\JupyterNotebook\[공모전]빅콘\DATASET(미세먼지)\환경기상데이터\노원구\{}.csv'.format(i[0])
        a = pd.read_csv(filepath)
        b = a['pm10'].replace([-999,-9999], np.NaN)
        b = a['noise'].replace([-999,-9999], np.NaN)
        b = a['temp'].replace([-999,-9999], np.NaN)
        b = a['humi'].replace([-999,-9999], np.NaN)
        b = a['pm25'].replace([-999,-9999], np.NaN)
        b.dropna(how='all',inplace=True)
        if len(b) == 0:
            pass
        else:
            df_return = pd.merge(df_return,a[['tm', 'pm10', 'pm25', 'temp', 'humi']],on = 'tm',how = 'outer')
    df_return['DONG_NM'] = no_dong[num]
    return df_return
    


### [함수] 최빈값(mode) 구하는 함수

In [0]:
# 최빈값 구하는 함수

def mode(gugan_num):
    
    d = {}
    num = list(map(str, gugan_num))
    
    for c in num:
        if c in d:
            d[c] += 1
        else:
            d[c] = 1
    
    return max(d, key=d.get)

### multi station 경우 적용예) 

In [0]:
# 윗부분 생략

# 행정동 - nowon1(공릉1동) - station 2개
# 행정동별 datetime변경 및 이상치 처리 

노원행정동1 = multi_st(1)

# tm 변수를 시간 변수로 바꾸고 인덱스로 설정해주기
노원행정동1['tm'] = pd.to_datetime(노원행정동1['tm'], format = '%Y%m%d%H%M')


#하고 싶은 것 :   
#동별 시간대별 미세먼지 농도
# -999, -9999 na로 바꾸고 해보자

노원행정동1['pm10_x'].replace([-999,-9999], np.NaN, inplace=True)
노원행정동1['pm10_y'].replace([-999,-9999], np.NaN, inplace=True)

노원행정동1['pm25_x'].replace([-999,-9999], np.NaN, inplace=True)   
노원행정동1['pm25_y'].replace([-999,-9999], np.NaN, inplace=True)   


# 시간데이터 분, 초만 잘라내서 'temp'라는 이름의 변수로 넣어줌
temp = [str(i)[:-6] for i in 노원행정동1.tm]
노원행정동1['temp'] = temp


# 미세먼지
# 같은 동 내 다른 스테이션에서 측정된 미세먼지농도를 하나의 리스트로 묶은 것 'mise'라는 이름의 변수를 가지는 데이터프레임 생성
tlist = []
for i in 노원행정동1[['temp','pm10_x', 'pm10_y']].values:  
    tlist.append([i[0],[i[1], i[2]]])

    
tdataframe = pd.DataFrame(tlist, columns = ['temp', 'mise'])
tdataframe.head()
no1 = pd.DataFrame(tdataframe.groupby('temp')['mise'].sum())


# 미세먼지 중앙값 구하기

mise_med = []

for i in no1.values:
    mise_med.append(s.median(i[0]))
no1['mise_med'] = mise_med


# 미세먼지 최빈값 구하기 
# 위와 같은 일을 하는데 1의자리에서 버린 값을 넣어줌 (최빈구간 구하기 위해)

gugan = []
for i in 노원행정동1[['temp', 'pm10_x', 'pm10_y']].values:
     gugan.append([i[0],[(i[1]//10)*10, (i[2]//10)*10]])


gugan_1 = pd.DataFrame(gugan, columns = ['temp', 'gugan']).groupby('temp')['gugan'].sum()
gugan_1.head()


# 전체 데이터에 적용
mise_mode = []
for i in range(len(gugan_1)):
  mise_mode.append(mode(gugan_1.values[i]))


# 초미세먼지
# 같은 동 내 다른 스테이션에서 측정된 미세먼지농도를 하나의 리스트로 묶은 것 'mise'라는 이름의 변수를 가지는 데이터프레임 생성

tlist = []
for i in 노원행정동1[['temp','pm25_x', 'pm25_y']].values:   
    tlist.append([i[0],[i[1], i[2]]])

tdataframe = pd.DataFrame(tlist, columns = ['temp', 'c_mise'])
tdataframe.head()
no1_1 = pd.DataFrame(tdataframe.groupby('temp')['c_mise'].sum())


# 초미세먼지 중앙값 구하기

mise_med = []

for i in no1_1.values:
    mise_med.append(s.median(i[0]))
no1_1['c_mise_med'] = mise_med


# 초미세먼지 최빈값 구하기
# 위와 같은 일을 하는데 1의자리에서 버린 값을 넣어줌 (최빈구간 구하기 위해)

gugan = []
for i in 노원행정동1[['temp','pm25_x', 'pm25_y']].values:
    gugan.append([i[0],[(i[1]//10)*10, (i[2]//10)*10]])
    
gugan_1 = pd.DataFrame(gugan, columns = ['temp', 'gugan']).groupby('temp')['gugan'].sum()
gugan_1


# 전체 데이터에 적용
cmise_mode = []
for i in range(len(gugan_1)):
  cmise_mode.append(mode(gugan_1.values[i]))

# 하나의 데이터프레임으로 만들기

no1.reset_index(inplace=True)
no1_1.reset_index(inplace=True)

no1_all = pd.merge(no1[['temp', 'mise_med']], no1_1[['temp','c_mise_med']], on = 'temp')

no1_all.columns = [['공릉1동','공릉1동','공릉1동'],['temp','mise_med','c_mise_med']]

no1_all['mise_mode'] = pd.DataFrame(mise_mode)

no1_all['c_mise_mode'] = pd.DataFrame(cmise_mode)

no1_all.columns = [['공릉1동','공릉1동','공릉1동','공릉1동','공릉1동'],['temp','mise_med','c_mise_med','mise_mode','c_mise_mode']]


no1_all.head()


NameError: name 'tm_all' is not defined

### single station 경우 적용예) 

In [0]:
# 윗 부분 생략

# 행정동 - nowon3(상계10동) - station 1개
# 행정동별 datetime변경 및 이상치 처리 


노원행정동3['tm'] = pd.to_datetime(노원행정동3['tm'], format = '%Y%m%d%H%M')


#하고 싶은 것 :   
#동별 시간대별 미세먼지 농도
# -999, -9999 na로 바꾸고 해보자

노원행정동3['pm10'].replace([-999,-9999], np.NaN, inplace=True)

노원행정동3['pm25'].replace([-999,-9999], np.NaN, inplace=True)   

노원행정동3.describe(include='all')

# 시간데이터 분, 초만 잘라내서 'temp'라는 이름의 변수로 넣어줌
temp = [str(i)[:-6] for i in 노원행정동3.tm]
노원행정동3['temp'] = temp


# 미세먼지
# 같은 동 내 다른 스테이션에서 측정된 미세먼지농도를 하나의 리스트로 묶은 것 'mise'라는 이름의 변수를 가지는 데이터프레임 생성

tlist = []
for i in 노원행정동3[['temp','pm10']].values:   # 미세먼지 데이터에서 시간(에서 분,초 뺀 것), 미세먼지만 떼어 냄
    tlist.append([i[0],i[1]])

노원행정동3_미세먼지 = pd.DataFrame(tlist, columns = ['temp', 'mise_med'])
노원행정동3_미세먼지['mise_med'].replace([-999,-9999], np.NaN, inplace=True)

# 미세먼지 중앙값 구하기

노원행정동3_median = 노원행정동3_미세먼지.pivot_table(index='temp',
                           aggfunc='median', 
                           values='mise_med')

no3 = pd.DataFrame(노원행정동3_median)


# 미세먼지 최빈값 구하기 
# 위와 같은 일을 하는데 1의자리에서 버린 값을 넣어줌 (최빈구간 구하기 위해)

gugan = []
for i in 노원행정동3[['temp', 'pm10']].values:
     gugan.append([i[0],[(i[1]//10)*10]])


gugan_3 = pd.DataFrame(gugan, columns = ['temp', 'gugan']).groupby('temp')['gugan'].sum()
gugan_3.head()



# 전체 데이터에 적용
mise_mode = []
for i in range(len(gugan_3)):
  mise_mode.append(mode(gugan_3.values[i]))


# 초미세먼지
# 같은 동 내 다른 스테이션에서 측정된 미세먼지농도를 하나의 리스트로 묶은 것 'mise'라는 이름의 변수를 가지는 데이터프레임 생성

tlist = []
for i in 노원행정동3[['temp','pm25']].values:   # 미세먼지 데이터에서 시간(에서 분,초 뺀 것), 미세먼지만 떼어 냄
    tlist.append([i[0],i[1]])


    
노원행정동3_초미세먼지 = pd.DataFrame(tlist, columns = ['temp', 'c_mise_med'])
노원행정동3_초미세먼지['c_mise_med'].replace([-999,-9999], np.NaN, inplace=True)


# 초미세먼지 중앙값 구하기    

노원행정동3_Cmedian = 노원행정동3_초미세먼지.pivot_table(index='temp',
                           aggfunc='median', 
                           values='c_mise_med')


no3_1 = pd.DataFrame(노원행정동3_Cmedian)


# 초미세먼지 최빈값 구하기 

gugan = []
for i in 노원행정동3[['temp', 'pm25']].values:
     gugan.append([i[0],[(i[1]//10)*10]])


gugan_3 = pd.DataFrame(gugan, columns = ['temp', 'gugan']).groupby('temp')['gugan'].sum()
gugan_3.head()



# 전체 데이터에 적용
cmise_mode = []
for i in range(len(gugan_3)):
  cmise_mode.append(mode(gugan_3.values[i]))


# 하나의 데이터프레임으로 만들기

no3.reset_index(inplace=True)
no3_1.reset_index(inplace=True)

no3_all = pd.merge(no3[['temp', 'mise_med']], no3_1[['temp','c_mise_med']], on = 'temp')

no3_all.columns = [['상계10동','상계10동','상계10동'],['temp','mise_med','c_mise_med']]

no3_all['mise_mode'] = pd.DataFrame(mise_mode)

no3_all['c_mise_mode'] = pd.DataFrame(cmise_mode)

no3_all.columns = [['상계10동','상계10동','상계10동','상계10동','상계10동'],['temp','mise_med','c_mise_med','mise_mode','c_mise_mode']]


no3_all.head()

